<a href="https://colab.research.google.com/github/shintaksr/Panting-Note-Detection-with-Compressive-Sensing-MFCC-and-SVM/blob/main/MFCC_CS_FINAL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import soundfile as sf
import librosa
import time

from scipy.fftpack import dct, idct
from sklearn.linear_model import OrthogonalMatchingPursuit as OMP

# **Load Data**

In [ ]:
nada_df = pd.read_csv("/content/drive/MyDrive/S1/EXTRACTED FEATURES IN CSV/nada_df.csv")
nada_df.head()

,Nada,Path,Nama
0,Do,/content/drive/MyDrive/S1/DATASET_SHINTA/Do/Do...,Do 2
1,Do,/content/drive/MyDrive/S1/DATASET_SHINTA/Do/Do...,Do 6
2,Do,/content/drive/MyDrive/S1/DATASET_SHINTA/Do/Do...,Do 3
3,Do,/content/drive/MyDrive/S1/DATASET_SHINTA/Do/Do...,Do 4
4,Do,/content/drive/MyDrive/S1/DATASET_SHINTA/Do/Do...,Do 9


# **Audio Processes**

## **Data Augmentation**

In [ ]:
def noise(signal):
  noise_amp = 0.035*np.random.uniform() * np.amax(signal)
  signal = signal + noise_amp*np.random.normal(size = signal.shape[0])
  return signal

def stretch(signal, rate=1.2):
  return librosa.effects.time_stretch(signal, rate)

def shift(signal):
  shift_range = int(np.random.uniform(low=-5, high=5)*1000)
  return np.roll(signal, shift_range)

def pitch(signal, sampling_rate, pitch_factor=0.7):
  pitched = librosa.effects.pitch_shift(signal,
                                       sampling_rate,
                                       pitch_factor)
  return pitched

## **Features Extraction with MFCC**

In [ ]:
def extract_mfcc(signal, fs):
  result = np.array([])

  # zcr
  zcr = np.mean(librosa.feature.zero_crossing_rate(y=signal).T, axis=0)
  result = np.hstack((result, zcr)) # stacking horizontally

  # rolloff
  rolloff = np.mean(librosa.feature.spectral_rolloff(y=signal, sr=fs, n_fft=120))
  result = np.hstack((result, rolloff))

  # mfcc
  mfcc = np.mean(librosa.feature.mfcc(y=signal, sr=fs, n_mfcc=20).T)
  result = np.hstack((result, mfcc))

  # MelSpectogram
  mel = np.mean(librosa.feature.melspectrogram(y=signal, sr=fs).T, axis=0)
  result = np.hstack((result, mel)) # stacking horizontally

  return result

## **Compressive Sensing - OMP**

In [ ]:
def cs_omp(x, M):
  
  x1 = dct(x, norm='ortho')

  m = M                                         # matriks A dengan ukuran (M x N)
  A = 2*(np.random.randn(m, x.shape[0]))-1
  A = A/(np.max(np.max(np.absolute(A))))      
  A = A/(np.linalg.norm(A, axis=0))            

  y = A @ x1

  omp = OMP(n_nonzero_coefs = 150, normalize=False)
  omp.fit(A,y)
  recon = omp.coef_

  xhat = idct(recon, norm='ortho')

  return xhat

## **Get Audio Features**

In [ ]:
def get_features(signal, fs) :
  # w/o augmentation
  res1 = extract_mfcc(signal, fs)
  result = np.array(res1)

  # w/ noise
  noise_sig = noise(signal)
  res2 = extract_mfcc(noise_sig, fs)
  result = np.vstack((result, res2))

  # data with stretching and pitching
  new_data = stretch(signal)
  data_stretch_pitch = pitch(signal, fs)
  res3 = extract_mfcc(data_stretch_pitch, fs)
  result = np.vstack((result, res3)) # stacking vertically

  return result

# **Proceed the audio**

## **Without CS-OMP**

In [ ]:
start_ori = time.time()

X = []
Y = []

for path, kelas, nada in zip(nada_df.Path, nada_df.Nada, nada_df.Nama):
  signal, fs = sf.read(path, start=0, stop=63000)
  #print('Fs = {} Hz'.format(fs))

  # MFCC
  feature = get_features(signal, fs)
  print('{}.wav has been extracted\n'.format(nada))

  for n in feature:
    X.append(n)
    Y.append(kelas)

end_ori = time.time()

Do 2.wav has been extracted

Do 6.wav has been extracted

Do 3.wav has been extracted

Do 4.wav has been extracted

Do 9.wav has been extracted

Do 7.wav has been extracted

Do 8.wav has been extracted

Do 1.wav has been extracted

Do 5.wav has been extracted

Do 10.wav has been extracted

Do 11.wav has been extracted

Do 12.wav has been extracted

Do 13.wav has been extracted

Do 14.wav has been extracted

Do 15.wav has been extracted

Do 16.wav has been extracted

Do 17.wav has been extracted

Do 18.wav has been extracted

Do 19.wav has been extracted

Do 20.wav has been extracted

Re 8.wav has been extracted

Re 4.wav has been extracted

Re 10.wav has been extracted

Re 2.wav has been extracted

Re 9.wav has been extracted

Re 6.wav has been extracted

Re 7.wav has been extracted

Re 1.wav has been extracted

Re 3.wav has been extracted

Re 5.wav has been extracted

Re 11.wav has been extracted

Re 12.wav has been extracted

Re 13.wav has been extracted

Re 14.wav has been extracted

In [ ]:
total_ori = end_ori - start_ori
print('Durasi Ekstrasi Ciri tanpa CS-OMP : {:.2f} Sekon'.format(float(total_ori)))

Durasi Ekstrasi Ciri tanpa CS-OMP : 43.70 Sekon


## **With CS-OMP**

In [ ]:
start_cs = time.time()
X = []
Y = []

for path, kelas, nada in zip(nada_df.Path, nada_df.Nada, nada_df.Nama):
  signal, fs = sf.read(path, start=0, stop=63000)
  
  # CS
  M = 6300                 # Ukuran matriks A
  xhat = cs_omp(signal, M)
  print('{}.wav has been Compressed'.format(nada))
  #print('Fs = {} Hz'.format(fs))

  # MFCC
  feature = get_features(xhat, fs)
  print('{}.wav has been extracted\n\n'.format(nada))

  for n in feature:
    X.append(n)
    Y.append(kelas)

end_cs = time.time()

Do 2.wav has been Compressed
Do 2.wav has been extracted


Do 6.wav has been Compressed
Do 6.wav has been extracted


Do 3.wav has been Compressed
Do 3.wav has been extracted


Do 4.wav has been Compressed
Do 4.wav has been extracted


Do 9.wav has been Compressed
Do 9.wav has been extracted


Do 7.wav has been Compressed
Do 7.wav has been extracted


Do 8.wav has been Compressed
Do 8.wav has been extracted


Do 1.wav has been Compressed
Do 1.wav has been extracted


Do 5.wav has been Compressed
Do 5.wav has been extracted


Do 10.wav has been Compressed
Do 10.wav has been extracted


Do 11.wav has been Compressed
Do 11.wav has been extracted


Do 12.wav has been Compressed
Do 12.wav has been extracted


Do 13.wav has been Compressed
Do 13.wav has been extracted


Do 14.wav has been Compressed
Do 14.wav has been extracted


Do 15.wav has been Compressed
Do 15.wav has been extracted


Do 16.wav has been Compressed
Do 16.wav has been extracted


Do 17.wav has been Compressed
Do 17.wav ha

In [ ]:
total_cs = end_cs - start_cs
print('Durasi Ekstrasi Ciri dengan CS-OMP : {:.2f} Sekon'.format(float(total_cs)))

Durasi Ekstrasi Ciri dengan CS-OMP : 8404.54 Sekon


# **Save features to CSV**

In [ ]:
Features = pd.DataFrame(X)
Features['labels'] = Y
Features.to_csv('features_original.csv', index=False)
Features.head()

,0,1,2,3,4,5,6,7,8,9,...,122,123,124,125,126,127,128,129,130,labels
0,0.019488,1638.709677,-7.653372,1.005586,1.318660,0.713475,0.411147,0.407605,0.269497,0.443340,...,0.000045,0.000083,0.000110,0.000037,0.000003,0.000001,1.406666e-07,3.826877e-08,9.145763e-09,Do
1,0.223243,18180.645161,-4.350660,1.067315,1.377500,0.757731,0.440129,0.456699,0.323801,0.488543,...,0.028725,0.029572,0.029567,0.030134,0.028039,0.030207,2.935766e-02,3.000341e-02,2.991015e-02,Do
2,0.022193,2038.709677,-8.946975,0.741895,0.961444,0.422105,0.215684,0.188944,0.159177,0.222867,...,0.000006,0.000014,0.000029,0.000045,0.000021,0.000002,3.239489e-07,2.111513e-08,2.736213e-09,Do
3,0.019330,1561.290323,-7.908329,0.939897,1.039002,0.538489,0.354722,0.749054,0.455866,0.660358,...,0.000081,0.000041,0.000029,0.000021,0.000006,0.000002,3.364003e-07,5.497470e-08,3.091877e-08,Do
4,0.192615,17267.741935,-4.715553,0.950845,1.073262,0.589858,0.385899,0.787217,0.485827,0.680339,...,0.024963,0.025899,0.026601,0.027020,0.026192,0.026099,2.579415e-02,2.642406e-02,2.584036e-02,Do
